Google
https://youtu.be/2wN3D0jsj9k?si=BnhtquabBGxq_hIA

You are given a transaction table, which records transactions between sellers and buyers. The structure of the table is as follows:
Transaction ID (INT), Customer_ID (INT) Amount (INT). Date (timestamp)

Every successful transaction will have two row entries into the table with two different transaction id but in ascending order sequence, the first one for the seller where their customer_id will be registered, and the second one for the buyer where their customer_id will be registered. The amount and date time for both will however be the same.

Write an sql query to find the 5 top seller-buyer combinations who have had maximum transactions between them.

Condition-Please disqualify the sellers who have acted as buyers and also the buyers who have acted as sellers for this condition.

In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext

In [2]:
spark = SparkSession.builder \
    .appName("application-1") \
    .enableHiveSupport() \
    .getOrCreate()

/usr/local/lib/python3.6/site-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


In [3]:
 %load_ext sparksql_magic

In [33]:
%%sparksql
DROP TABLE transactions;


In [34]:
%%sparksql

CREATE TABLE transactions (
  transaction_id INT,
  customer_id INT,
  amount INT,
  tran_Date STRING
) USING PARQUET;

In [35]:
%%sparksql


INSERT INTO Transactions VALUES 
    (1, 101, 500, '2025-01-01 10:00:01') 
  ,(2, 201, 500, '2025-01-01 10:00:01') 
  ,(3, 102, 300, '2025-01-02 00:50:01') 
  ,(4, 202, 300, '2025-01-02 00:50:01') 
  ,(5, 101, 700, '2025-01-03 06:00:01') 
  ,(6, 202, 700, '2025-01-03 06:00:01') 
  ,(7, 103, 200, '2025-01-04 03:00:01') 
  ,(8, 203, 200, '2025-01-04 03:00:01') 
  ,(9, 101, 400, '2025-01-05 00:10:01') 
  ,(10, 201, 400, '2025-01-05 00:10:01') 
  ,(11, 101, 500, '2025-01-07 10:10:01') 
  ,(12, 201, 500, '2025-01-07 10:10:01') 
  ,(13, 102, 200, '2025-01-03 10:50:01') 
  ,(14, 202, 200, '2025-01-03 10:50:01') 
  ,(15, 103, 500, '2025-01-01 11:00:01') 
  ,(16, 101, 500, '2025-01-01 11:00:01')
  ,(17, 203, 200, '2025-11-01 11:00:01') 
  ,(18, 201, 200, '2025-11-01 11:00:01')

In [36]:
%%sparksql
select * from transactions

transaction_id,customer_id,amount,tran_Date
14,202,200,2025-01-03 10:50:01
15,103,500,2025-01-01 11:00:01
16,101,500,2025-01-01 11:00:01
17,203,200,2025-11-01 11:00:01
18,201,200,2025-11-01 11:00:01
5,101,700,2025-01-03 06:00:01
6,202,700,2025-01-03 06:00:01
7,103,200,2025-01-04 03:00:01
8,203,200,2025-01-04 03:00:01
9,101,400,2025-01-05 00:10:01


In [37]:
%%sparksql

WITH CTE as (
  select *, LEAD(customer_id, 1) OVER(ORDER BY transaction_id) as buyer_id
  FROM
  transactions
),
final as (
  select * from CTE where transaction_id % 2 = 1
),
combination as (
  select customer_id, buyer_id, count(*)
  FROM
  final
  group by customer_id, buyer_id
),
fraud_customers as (
  select customer_id as fraud FROM combination
  INTERSECT
  select buyer_id FROM combination
)
select *
from combination
where customer_id not in (select fraud from fraud_customers)
and buyer_id not in (select fraud from fraud_customers)

customer_id,buyer_id,count(1)
102,202,2
